In [1]:
%pwd

'c:\\Users\\hp\\OneDrive\\Desktop\\legalconnect-gh\\legalconnect-gh\\chatbot\\research'

In [2]:
import os
os.chdir('../')

In [3]:
%pwd

'c:\\Users\\hp\\OneDrive\\Desktop\\legalconnect-gh\\legalconnect-gh\\chatbot'

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
#Extarct Data From PDF File
def load_pdf_file(data):
    loader = DirectoryLoader(data, 
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents

In [6]:
%pwd

'c:\\Users\\hp\\OneDrive\\Desktop\\legalconnect-gh\\legalconnect-gh\\chatbot'

In [7]:
extracted_data = load_pdf_file("Data/")

In [8]:
#Split Data into Chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20
    )
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [9]:
text_chunks = text_split(extracted_data)
print("Length of text chunks: ", len(text_chunks))

Length of text chunks:  5562


In [10]:
from langchain.llms import LlamaCpp

llm = LlamaCpp(
    model_path="C:\\Users\\hp\\OneDrive\\Desktop\\legalconnect-gh\\legalconnect-gh\\chatbot\\mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    temperature=0.7,
    max_tokens=256,
    top_p=1,
    n_ctx=2048,  # Safer default; check metadata if you want 4096
    n_threads=8,          # Set based on your CPU (see notes below)
    n_batch=128,          # Larger batch improves throughput  
    verbose=True,
)


llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from C:\Users\hp\OneDrive\Desktop\legalconnect-gh\legalconnect-gh\chatbot\mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32       

In [11]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [12]:
embeddings = download_hugging_face_embeddings()

C:\Users\hp\AppData\Local\Temp\ipykernel_16968\502912814.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\hp\anaconda3\envs\legalconnect-bot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=text_chunks, embedding=embeddings, persist_directory="./db")

In [14]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)


In [15]:
template = """
You are LegalConnectBot, an expert Ghanaian legal assistant. You only answer questions with verified legal knowledge from Ghana or relevant legal documents. Be accurate and formal. If unsure, say "I don't know".

Context:
{context}

User Question:
{question}

Helpful Answer:
"""


In [16]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

In [22]:
query = "How does the Vice-President of Ghana succeed the Presidency in case of vacancy or incapacity?"
result = qa_chain(query)

print("Answer:", result['result'])


Llama.generate: 45 prefix-match hit, remaining 402 prompt tokens to eval
llama_perf_context_print:        load time =   50440.62 ms
llama_perf_context_print: prompt eval time =   35875.70 ms /   402 tokens (   89.24 ms per token,    11.21 tokens per second)
llama_perf_context_print:        eval time =   24434.33 ms /    67 runs   (  364.69 ms per token,     2.74 tokens per second)
llama_perf_context_print:       total time =   60427.85 ms /   469 tokens


Answer:  If the President of Ghana exceeds half his term, the Vice-President is only eligible to serve one full term as President. Additionally, if the President is absent from Ghana or unable to perform his functions, the Vice-President performs the functions of the President until the President returns or becomes able to do so.


In [18]:
print(result)

{'query': 'How can I register a business in Ghana?', 'result': ' To register a business in Ghana, you will need to comply with the laws and regulations set forth by the Electoral Commission. You can begin by providing a copy of your Constitution and the names and addresses of your national officers. Additionally, you must demonstrate that there is at least one ordinarily resident, or registered as a voter, in each district of Ghana. Finally, you should ensure that your business achieves reasonable regional and gender balance in recruitment and appointment to public offices, provides adequate facilities for free mobility of people, goods and services throughout Ghana, and satisfies any other requirements set forth by the Electoral Commission.', 'source_documents': [Document(metadata={'page_label': '52', 'subject': '', 'author': 'abrsaa', 'title': 'Ghana Constitution', 'moddate': '2009-02-24T14:55:39+01:00', 'creationdate': '2009-02-24T14:55:39+01:00', 'page': 51, 'keywords': '', 'source

In [19]:
len(result)

3